In [106]:
# from PyPaperBot import __main__ as pypaper

In [141]:
COLLECTION_NAME = "pdf"
FOLDER_PATH="./documents"
MODEL_NAME="Llama3-8b-8192"
topic="quantum computing"
year=2023
pagine_scholar=1


In [143]:
import subprocess
print(["powershell", "python", "-m", "PyPaperBot", "--query={}".format(topic.replace(" ", "+")), "--scholar-pages={}".format(pagine_scholar), "--min-year={}".format(year), "--dwn-dir={}".format(FOLDER_PATH), "--restrict=1"])

['powershell', 'python', '-m', 'PyPaperBot', '--query=quantum+computing', '--scholar-pages=1', '--min-year=2023', '--dwn-dir=./documents', '--restrict=1']


In [145]:
!python -m PyPaperBot --query=quantum+computing --scholar-pages=1 --scholar-results=1 --min-year=2023 --dwn-dir=./documents --restrict=1

PyPaperBot is a Python tool for downloading scientific papers using Google Scholar, Crossref and SciHub.
If you like this project, you can give me a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)

Scholar results best applied along with --scholar-pages=1
Query: quantum+computing

Google Scholar page 1 : 1 papers found
Searching paper 1 of 1 on Crossref...
Papers found on Crossref: 1/1


Using https://sci-hub.ee as Sci-Hub instance
Download 1 of 1 -> A survey of important issues in quantum computing and communications

Work completed!
If you like this project, you can offer me a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)



In [107]:
# #https://github.com/ferru97/PyPaperBot/tree/master/PyPaperBot

# pypaper.start(
#     query="Quantum Computing", 
#     scholar_pages=[1], 
#     min_date=2018, 
#     dwn_dir="./documents", 
#     # SciHub_URL="https://sci-hub.do", 
#     proxy=[], 
#     scholar_results=1
# )

In [109]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv(override=True)

from utils import database_managers, embedding, text_processing

In [110]:
#print your results with Markup language
def print_result(result):
  output_text = f"""
  ### Answer: 
  {result['answer']}
  ### Sources: 
  {result['sources']}
  ### All relevant sources:
  {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
  """
  return(output_text)

In [111]:
COLLECTION_NAME = "pdf"

In [112]:
text_splitter=text_processing.TextSplitter()

In [113]:
embedding = embedding.EmbeddingFunction('fast-bgeEmbedding').embedder

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 2436.28it/s]


In [114]:
vectore_store=qdrantClient = database_managers.QDrantDBManager(
    url=os.getenv('QDRANT_URL'),
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=768,
    embedding=embedding,
    record_manager_url=r"sqlite:///record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store
retriever = vectore_store_client.as_retriever()

Collection pdf already exists!


In [115]:
#iterate through the pdf in the folder 
import os

folder_path="./documents"
docs=[]

for pdf_file in os.listdir(folder_path):
    if pdf_file.endswith('.pdf'):

        loader = PyPDFLoader(os.path.join(folder_path, pdf_file))
        docs=docs+loader.load_and_split()

In [116]:
len(docs)

131

In [117]:
docs=text_splitter.fixed_split(docs)

In [118]:
len(docs)

241

In [124]:
vectore_store.index_documents(docs)

Prompt engineering

In [120]:

#create custom prompt for your use case
prompt_template="""Sei Debbie, un'assistente alla ricerca AI che parla ai ricercatori universitari. 

Rispondi alle domande utilizzando i fatti forniti. 
Usa sempre cortesie e saluti gentili come "Buongiorno" e "Buon pomeriggio". 
Utilizza i seguenti pezzi di contesto per rispondere alla domanda degli utenti.
Prendi nota delle fonti e includile nella risposta nel formato: "SOURCES: source1 source2", usa "SOURCES" in maiuscolo indipendentemente dal numero di fonti.

Traduci sempre tutto in italiano.

Se non conosci la risposta, dì semplicemente "Non lo so", non cercare di inventare una risposta.
----------------
{summaries}
"""

messages = [
    SystemMessagePromptTemplate.from_template(prompt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

chain_type_kwargs = {"prompt": prompt}

In [121]:
llm = ChatGroq(temperature=0, model_name="Llama3-8b-8192")

In [122]:

#build your chain for RAG+C
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [123]:
query = "cos'è l'adiabatic quantum computing?"
result = chain(query)
display(Markdown(print_result(result)))


  ### Answer: 
  Buongiorno! L'adiabatic quantum computing, o AQC, è un approccio alternativo al modello standard di calcolo quantistico, che si basa sull'idea di evolvere gradualmente un sistema quantistico in modo da raggiungere lo stato finale desiderato. Questo approccio è stato proposto per la prima volta nel 2000 da Seth Lloyd e da altri ricercatori.

In AQC, il sistema quantistico è descritto da un'hamiltoniana, che è un operatore matematico che descrive l'energia del sistema. L'obiettivo è quello di evolvere l'hamiltoniana in modo da raggiungere lo stato finale desiderato, che è il minimo di energia del sistema.

L'adiabatic quantum computing è stato dimostrato essere un approccio universale per la computazione quantistica, cioè può eseguire qualsiasi calcolo quantistico che può essere eseguito con il modello standard di calcolo quantistico. Inoltre, AQC ha anche mostrato di avere un vantaggio quantistico rispetto ai calcoli classici, cioè può eseguire certi calcoli più velocemente di quanto non possa fare un computer classico.


  ### Sources: 
  ./documents\Adiabatic quantum computation.pdf
  ### All relevant sources:
  ./documents\Adiabatic quantum computation.pdf
  

# With documents download on chain with agents

In [134]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents import load_tools
from langchain.agents import Tool
memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output",return_messages=True) 


In [125]:
from langchain.agents.load_tools import get_all_tool_names
print(get_all_tool_names())

['sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'read_file', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal']


In [131]:
!pip install -U duckduckgo-search

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.6 MB 12.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.2/2.6 MB 15.9 MB/s eta 0:00:01
   -------------------------------------- - 2.5/2.6 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3


In [135]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [136]:
agent = initialize_agent(
        agent="zero-shot-react-description",
        tools=tools,
        llm=llm,
        memory=memory,
        return_intermediate_steps= True, # Make sure you set it to True
        verbose=True
    )

c:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-agent-chatbot\.venv\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [137]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

search: useful for when you need to answer questions about current events. You should ask targeted questions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


https://medium.com/@kbdhunga/exploring-langchain-chains-agents-a-quick-overview-9d0a8c4d7ba0

In [ ]:
#custom tool 

def embed_papers(question : str) -> str:
    pass
    #TODO

embed_papers_tool = Tool(
    name="Embed papers",
    func=embed_papers,
    description="Utile per cercare documenti relativi a paper non ancora ricercati durante le precedenti ricerche"
)